In [1]:
import pandas as pd
import numpy as np

In [2]:
df_index= pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level.xlsx')
df_index.head()

,date,index_level
0,2010-04-16,3356.332
1,2010-04-19,3176.423
2,2010-04-20,3173.374
3,2010-04-21,3236.679
4,2010-04-22,3201.541


In [3]:
df_2008 = pd.read_excel('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/CSI300_Index_Level(2008-3-31).xlsx')
df_2008.head()



,指标名称,沪深300指数
0,频率,日
1,单位,点
2,指标ID,M002845714
3,2022-12-30,3871.63
4,2022-12-29,3856.7


In [4]:
a = ['2010-04-15',3394.57]
df_index2 = pd.DataFrame(np.insert(df_index.values,0,a,axis = 0))
df_index2.columns = df_index.columns
df_index2

,date,index_level
0,2010-04-15,3394.57
1,2010-04-16,3356.332
2,2010-04-19,3176.423
3,2010-04-20,3173.374
4,2010-04-21,3236.679
...,...,...
3087,2022-12-26,3843.4887
3088,2022-12-27,3887.854
3089,2022-12-28,3871.2644
3090,2022-12-29,3856.7007


In [5]:
diff = np.array(df_index2["index_level"])
diff = np.diff(diff)
diff

array([-38.238000000000284, -179.9090000000001, -3.048999999999978, ...,
       -16.58959999999979, -14.563700000000154, 14.933100000000195],
      dtype=object)

In [6]:
df_index3 = pd.DataFrame(np.insert(df_index.values,2,diff,axis = 1))
df_index3.columns = ['date','index_level','diff']
df_index3['date']=pd.to_datetime(df_index3['date'])
df_index3

,date,index_level,diff
0,2010-04-16,3356.332,-38.238
1,2010-04-19,3176.423,-179.909
2,2010-04-20,3173.374,-3.049
3,2010-04-21,3236.679,63.305
4,2010-04-22,3201.541,-35.138
...,...,...,...
3086,2022-12-26,3843.4887,15.2699
3087,2022-12-27,3887.854,44.3653
3088,2022-12-28,3871.2644,-16.5896
3089,2022-12-29,3856.7007,-14.5637


In [7]:
df_index3.set_index("date", inplace=True)
df_index3


,index_level,diff
date,,
2010-04-16,3356.332,-38.238
2010-04-19,3176.423,-179.909
2010-04-20,3173.374,-3.049
2010-04-21,3236.679,63.305
2010-04-22,3201.541,-35.138
...,...,...
2022-12-26,3843.4887,15.2699
2022-12-27,3887.854,44.3653
2022-12-28,3871.2644,-16.5896


In [8]:
df_index4 = df_index3.drop('index_level',axis = 1)
df_index4
#check missing value
df_index4.isnull().sum()
#find out missing value
df_index4[df_index4.isnull().values==True]
df_index4.dropna(inplace=True)
df_index4

,diff
date,
2010-04-16,-38.238
2010-04-19,-179.909
2010-04-20,-3.049
2010-04-21,63.305
2010-04-22,-35.138
...,...
2022-12-26,15.2699
2022-12-27,44.3653
2022-12-28,-16.5896


In [9]:
df_index4['binary'] = df_index4['diff'].apply(lambda x: 1 if x > 0 else 0)
df_index4

,diff,binary
date,,
2010-04-16,-38.238,0
2010-04-19,-179.909,0
2010-04-20,-3.049,0
2010-04-21,63.305,1
2010-04-22,-35.138,0
...,...,...
2022-12-26,15.2699,1
2022-12-27,44.3653,1
2022-12-28,-16.5896,0


In [10]:
monthly_groups=df_index4.groupby([pd.Grouper(freq='M'),'binary']).mean()
monthly_groups

diff
date       binary           
2010-04-30 0      -44.201222
           1       35.303000
2010-05-31 0      -60.790909
           1       41.622111
2010-06-30 0      -34.557417
...                      ...
2022-10-31 1       37.311480
2022-11-30 0      -27.561730
           1       51.662533
2022-12-31 0      -25.321020
           1       22.650625

[306 rows x 1 columns]

In [11]:
df = monthly_groups
df

diff
date       binary           
2010-04-30 0      -44.201222
           1       35.303000
2010-05-31 0      -60.790909
           1       41.622111
2010-06-30 0      -34.557417
...                      ...
2022-10-31 1       37.311480
2022-11-30 0      -27.561730
           1       51.662533
2022-12-31 0      -25.321020
           1       22.650625

[306 rows x 1 columns]

In [12]:
grouped = df.groupby('date')['diff']
diff1 = grouped.apply(lambda x: x.loc[x.index.get_level_values('binary') == 1].sum() / x.loc[x.index.get_level_values('binary') == 0].sum() if 0 in x.index.get_level_values('binary') and 1 in x.index.get_level_values('binary') else '+' if 1 in x.index.get_level_values('binary') else '-').to_numpy()
diff1

array([-0.79868832, -0.68467657, -0.84536329, -1.28588455, -0.76201036,
       -1.02899363, -2.44128302, -0.77703127, -1.06541659, -0.71269341,
       -0.98300817, -1.03680044, -0.98263065, -0.6214653 , -0.86493997,
       -0.67064246, -0.89523734, -1.05014159, -1.54017823, -0.63338794,
       -0.89518406, -1.67857863, -1.12104451, -0.75566699, -1.9697871 ,
       -1.22735396, -1.01852182, -0.50263197, -0.67043503, -1.13749611,
       -0.76605664, -1.18578238, -1.95486998, -1.38234787, -0.62985578,
       -0.76668741, -2.0060542 , -1.01419653, -0.50424   , -0.89676571,
       -2.21264738, -1.11004001, -1.0516669 , -1.48373571, -0.84907867,
       -0.8183738 , -0.68689814, -1.36739171, -1.18131474, -1.1999226 ,
       -1.30272632, -1.95700897, -1.02622184, -0.94514883, -1.74975072,
       -3.19231448, -0.83207011, -0.83333585, -0.77796449, -1.40176126,
       -1.35066859, -0.90522449, -0.83486896, -0.67733474, -0.71890787,
       -0.74118689, -0.78382012, -1.17767333, -1.10056087, -0.37

In [13]:
diff2 = list(diff1)
diff2

[-0.7986883218412777,
 -0.6846765697954569,
 -0.8453632894599488,
 -1.2858845480380032,
 -0.7620103580437214,
 -1.0289936271906528,
 -2.4412830162621275,
 -0.7770312747042021,
 -1.0654165872217507,
 -0.7126934096837159,
 -0.9830081700604921,
 -1.0368004363011172,
 -0.9826306526252427,
 -0.6214652974247107,
 -0.8649399713188294,
 -0.6706424630952941,
 -0.8952373379584185,
 -1.0501415854953555,
 -1.5401782328403473,
 -0.6333879447365215,
 -0.8951840592235258,
 -1.6785786325921188,
 -1.1210445120346266,
 -0.7556669932945225,
 -1.969787098824281,
 -1.2273539630709869,
 -1.018521824806419,
 -0.5026319654327479,
 -0.6704350259623848,
 -1.137496114832875,
 -0.7660566435489303,
 -1.185782379676197,
 -1.9548699849904707,
 -1.3823478691752373,
 -0.6298557844411229,
 -0.7666874099435801,
 -2.006054201889607,
 -1.0141965295187554,
 -0.5042399965022398,
 -0.8967657083580323,
 -2.212647377054899,
 -1.1100400111659077,
 -1.0516668987731306,
 -1.48373570656883,
 -0.8490786747784836,
 -0.81837379868487

In [14]:
diff3 = []
for i in diff2:
    if i != '+' and i != '-':
        i = abs(float(i))
        i = 100*(1-1/(1+i))
    diff3.append(i)
diff3

[44.403931028121534,
 40.64142530803916,
 45.81012824349329,
 56.253258684551255,
 43.246644638896804,
 50.71448295357136,
 70.94107066247095,
 43.72637025386871,
 51.5836172621656,
 41.612433705535736,
 49.571564298219975,
 50.90338836454562,
 49.56196209940167,
 38.32738809839174,
 46.37897115300553,
 40.14278805369024,
 47.23615982169195,
 51.222881040268256,
 60.63268368055294,
 38.77755721031062,
 47.23467648784947,
 62.6667670744361,
 52.85341753432873,
 43.04159024351808,
 66.32755255769366,
 55.103678329543996,
 50.45879674370614,
 33.45010468268544,
 40.135354895119505,
 53.21628923390173,
 43.37667460141722,
 54.249791319658236,
 66.15756344341408,
 58.02460199289883,
 38.644878304806596,
 43.39689101922476,
 66.73380009677139,
 50.35241172623177,
 33.521246455002704,
 47.27867571658773,
 68.87302331584486,
 52.60753375726522,
 51.25914442554068,
 59.738067244624204,
 45.919012877059,
 45.00580679708197,
 40.71959807296878,
 57.759419457996785,
 54.15608836805488,
 54.5438554

In [15]:
diff4 = []
for i in diff3:
    if i == '-':
        i = int('0')
    if i =='+':
        i = int('100')
    diff4.append(i)
diff4

[44.403931028121534,
 40.64142530803916,
 45.81012824349329,
 56.253258684551255,
 43.246644638896804,
 50.71448295357136,
 70.94107066247095,
 43.72637025386871,
 51.5836172621656,
 41.612433705535736,
 49.571564298219975,
 50.90338836454562,
 49.56196209940167,
 38.32738809839174,
 46.37897115300553,
 40.14278805369024,
 47.23615982169195,
 51.222881040268256,
 60.63268368055294,
 38.77755721031062,
 47.23467648784947,
 62.6667670744361,
 52.85341753432873,
 43.04159024351808,
 66.32755255769366,
 55.103678329543996,
 50.45879674370614,
 33.45010468268544,
 40.135354895119505,
 53.21628923390173,
 43.37667460141722,
 54.249791319658236,
 66.15756344341408,
 58.02460199289883,
 38.644878304806596,
 43.39689101922476,
 66.73380009677139,
 50.35241172623177,
 33.521246455002704,
 47.27867571658773,
 68.87302331584486,
 52.60753375726522,
 51.25914442554068,
 59.738067244624204,
 45.919012877059,
 45.00580679708197,
 40.71959807296878,
 57.759419457996785,
 54.15608836805488,
 54.5438554

In [19]:
df_index5=df_index4.groupby([pd.Grouper(freq='M')]).mean()
df_index5
df_index5.dropna(inplace=True)
df_index5['RSI'] = diff4
df_index5 = df_index5.drop('binary',axis =1)
df_index5

,RSI
date,
2010-04-30,44.403931
2010-05-31,40.641425
2010-06-30,45.810128
2010-07-31,56.253259
2010-08-31,43.246645
...,...
2022-08-31,54.523974
2022-09-30,49.217163
2022-10-31,45.951869


In [21]:
df_index5.to_csv('C:/Users/XLL/Desktop/数据/CSI300_Index & Future_0521Final/CSI300_Index & Future/Monthly_RSI.csv', index = False, header=True)